In [66]:
# Load table into memory

from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
import pandas as pd 
sql = """
select * from tt_h.anomalies_tfl_json
where tfl_message = 'ok'
"""
df = pd.read_sql(sql, conn)

In [106]:
# Parse the above 
import json

for r in df.iterrows():
    
    index = r[0]
    row = r[1]
    
    tfl_response = row["tfl_response"]
    j1 = json.loads(tfl_response)
    journeys = j1["journeys"]

    # Keep only the fastest journey
    d = 1000
    for j in journeys:
        
        # Filter out taxi journeys
        con = False
        for l in j["legs"]:
            if l["mode"]["name"] == "taxi":
                con = True
        if con:
            continue
        
        if j['duration'] < d:
            d = j['duration']
            fastest_journey = j

    df.loc[index, "duration"] = fastest_journey["duration"]
    df.loc[index, "fastest_journey_arrival_icscode"] = fastest_journey["legs"][-1]["arrivalPoint"]["icsCode"]
    df.loc[index, "fastest_journey_json"] = json.dumps(fastest_journey )

In [107]:
df2 = df[df["station_name_arrive"] != "Stratford"]
df2 = df2.sort_values(["station_name_depart", "duration"]).drop_duplicates("station_name_depart")

In [108]:
df2 = df2[["icscode_depart", "station_name_depart", "icscode_arrive", "station_name_arrive", "duration", "fastest_journey_arrival_icscode", "fastest_journey_json"]]
df2[df2.station_name_depart.str.contains("ewport")]

,icscode_depart,station_name_depart,icscode_arrive,station_name_arrive,duration,fastest_journey_arrival_icscode,fastest_journey_json
450,1000647,Newport (Essex),1000138,Liverpool Street,62.0,1000138,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."


In [34]:
sql = """
select * from tt_h.parsed_rail_journeys
where natrail_journey_minutes is null limit 1
"""

pd.read_sql(sql, conn)

,nlc,icscode,station_name,tfl_message,tfl_dest,depart_lat,depart_lng,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime,id
0,506,1000006,Amersham,ok,E145HP,51.674204,-0.607596,Amersham Rail Station → Chalfont & Latimer Rai...,None,None,None,None,None,None,None,None,41


In [35]:
# Get 
sql = """
select id from tt_h.parsed_rail_journeys
where natrail_journey_minutes is null
"""

problem_stations = pd.read_sql(sql, conn)
ids = [int(i) for i in problem_stations["id"]]


In [110]:
# Open each id and add 
import parse_tfl_json
from process_journeys import  add_cycle_and_total_time

Journeys = Automapped_Base.classes.parsed_rail_journeys

for id in ids:

    parsed_journey = session.query(Journeys).filter(Journeys.id == id).one()

    try:
        anomaly_row = df2[df2["icscode_depart"] == parsed_journey.icscode].iloc[0]
    except:
        print("Journey: from {} to {}".format(parsed_journey.station_name, parsed_journey.tfl_dest))
        print("No clondon found")
        continue
    
    

    journey = json.loads(anomaly_row["fastest_journey_json"])

    try:
        journeys = add_cycle_and_total_time([journey], parsed_journey.tfl_dest)
    except:
        pass
        print("CLondond: {}".format(anomaly_row["station_name_arrive"]))
        print("Couldn't find cycle time for {} {} {}".format(parsed_journey.station_name,anomaly_row["station_name_arrive"], parsed_journey.tfl_dest))
        continue
        
    journey = journeys[0]

    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    num_changes = parse_tfl_json.num_changes(legs)
    final_arrival = parse_tfl_json.final_arrival(legs)
    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

    parsed_journey.arrive_lat = lat_lng["arrive"]["lat"]
    parsed_journey.arrive_lng = lat_lng["arrive"]["lng"]
    parsed_journey.cycle_minutes = journey["cycle_minutes"]
    parsed_journey.cycle_miles = journey["cycle_miles"]
    parsed_journey.total_journeytime = travel_time_minutes + journey["cycle_minutes"]
    parsed_journey.natrail_journey_summary = summary
    parsed_journey.natrail_journey_minutes = travel_time_minutes
    parsed_journey.final_arrival = final_arrival
    parsed_journey.natrail_train_changes = num_changes
    
    if id == 1208:
        
        print("---")
        print(summary)
        print(parsed_journey.natrail_journey_minutes)
        print("----")
    
    session.add(parsed_journey)
    session.commit()

CLondond: Paddington
Couldn't find cycle time for Amersham Paddington E145HP
Journey: from Appledore to E145HP
No clondon found
CLondond: Paddington
Couldn't find cycle time for Aylesbury Paddington E145HP
CLondond: Paddington
Couldn't find cycle time for Aylesbury Vale Parkway Paddington E145HP
CLondond: Paddington
Couldn't find cycle time for Aylesbury Vale Parkway Paddington SW1H9AJ
CLondond: King's Cross
Couldn't find cycle time for Bayford King's Cross SW1H9AJ
CLondond: King's Cross
Couldn't find cycle time for Bush Hill Park King's Cross E145HP
CLondond: King's Cross
Couldn't find cycle time for Bush Hill Park King's Cross SW1H9AJ
CLondond: King's Cross
Couldn't find cycle time for Chalfont & Latimer King's Cross E145HP
CLondond: King's Cross
Couldn't find cycle time for Chalfont & Latimer King's Cross SW1H9AJ
Journey: from Chartham to E145HP
No clondon found
Journey: from Chilham to E145HP
No clondon found
CLondond: King's Cross
Couldn't find cycle time for Chorleywood King's Cr